<a href="https://colab.research.google.com/github/tonycico0912/ChatGPT_Python/blob/main/pdr_ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 라이브러리를 가져옵니다.
import requests
from PublicDataReader import TransactionPrice
import logging
import PublicDataReader as pdr
from google.colab import files
import datetime

# 로깅 레벨을 INFO로 설정합니다. 이렇게 하면 프로그램에서 발생하는 정보성 메시지를 콘솔에 출력합니다.
logging.basicConfig(level=logging.INFO)

# Google Colab에서 파일을 업로드합니다.
uploaded = files.upload()

# API 키를 읽어옵니다. 이 키는 data_go_kr_api_key.txt 파일에 저장되어 있습니다.
logging.info("Reading API key...")
with open('data_go_kr_api_key.txt', 'r', encoding='utf-8') as file:
    service_key = file.readline().strip()

# TransactionPrice API를 초기화합니다.
logging.info("Initializing API...")
api = TransactionPrice(service_key)

# 부동산 유형을 나타내는 리스트를 생성합니다.
property_types = ["아파트", "오피스텔", "단독다가구", "연립다세대", "토지", "분양입주권", "공장창고"]

# 거래 유형을 나타내는 리스트를 생성합니다.
trade_types = ["매매", "전월세"]

# 사용자에게 부동산 유형을 선택하도록 요청합니다.
logging.info("Asking user to select property_type...")
print("[Q1] 원하는 property_type을 선택해주세요:")
for i, property_type in enumerate(property_types, 1):
    print(f"{i}. {property_type}")

# 사용자의 선택을 받습니다.
user_choice = int(input("선택(1-7): ")) - 1
selected_property_type = property_types[user_choice]

print("----------------------------------------------------------------")

# 사용자에게 거래 유형을 선택하도록 요청합니다.
logging.info("Asking user to select trade_type...")
print("[Q2] 원하는 trade_type을 선택해주세요:")
for i, trade_type in enumerate(trade_types, 1):
    print(f"{i}. {trade_type}")

# 사용자의 선택을 받습니다.
user_choice = int(input("선택(1-2): ")) - 1
selected_trade_type = trade_types[user_choice]

print("----------------------------------------------------------------")

# 사용자로부터 시군구명을 입력받습니다.
sigungu_name = input("시군구명을 입력하세요: ")

# 법정동 코드를 가져옵니다.
code = pdr.code_bdong()

# 입력한 시군구명이 포함된 행을 찾습니다.
result = code.loc[(code['시군구명'].str.contains(sigungu_name)) & (code['읍면동명']=='')]

# 원하는 열만 선택하여 출력합니다.
result = result[['시도코드', '시도명', '시군구코드', '시군구명', '법정동코드', '읍면동명', '동리명', '생성일자', '말소일자']]
print(result)

# 시군구코드를 추출합니다.
sigungu_code = str(result['시군구코드'].values[0])

print("[Q4] 원하는 조회기간을 입력해주세요:")

# 현재 년월을 얻습니다.
current_year_month = datetime.datetime.now().strftime("%Y%m")

while True:
    # 사용자로부터 조회할 년월을 입력받습니다.
    year_month = input("조회할 년월을 입력하세요(예: 202212): ")

    # 입력된 년월이 현재보다 뒤인지 확인합니다. 뒤라면 다시 입력을 요청합니다.
    if int(year_month) > int(current_year_month):
        print("입력된 년월이 현재보다 뒤입니다. 다시 입력해주세요.")
    else:
        break

# 단일 월에 대한 데이터를 조회합니다.
logging.info("Retrieving data for single month...")
df = api.get_data(
        property_type=selected_property_type,
        trade_type=selected_trade_type,
        sigungu_code=sigungu_code,
        year_month=year_month,
    )

# 데이터의 마지막 몇 행을 출력합니다.
logging.info("Displaying the last few rows of the data...")
df.tail()
